In [17]:
from llama_index import VectorStoreIndex, SimpleDirectoryReader, StorageContext, load_index_from_storage
import os
import openai


In [14]:
os.environ['OPENAI_API_KEY'] = ''
openai.api_key = ''


In [8]:
# Loading from a directory
documents = SimpleDirectoryReader('data').load_data()


In [19]:
# Construct a simple vector index
index = VectorStoreIndex.from_documents(documents)


In [20]:
# Saving Index for future use
index.storage_context.persist()

# Loading Index from local storage
storage_context = StorageContext.from_defaults(persist_dir="./storage")
index = load_index_from_storage(storage_context)


In [24]:
# Querying the index
query_engine = index.as_query_engine()
response = query_engine.query("What blockchains does Dune support?")
print(response)



Dune supports Ethereum, Polygon, Goerli, Optimism, Solana, and Bitcoin.
